Test with Github

In [ ]:
# Set up colab instance
from google.colab import drive
drive.mount('/content/drive')

!pip3 install pydicom
!git clone https://github.com/thomasp05/gif-705-projet

import os
os.chdir('gif-705-projet')

In [ ]:
import time

import torch

from dataset import *
from models import *

torch.manual_seed(111)

In [ ]:
# HYPERPARAMETERS
N_EPOCH = 10
BATCH_SIZE = 64

In [ ]:
dataset = dcm_dataset('../drive/MyDrive/gif-7005')

train_set, test_set = train_test_split(dataset)

train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=BATCH_SIZE, num_workers=2)
test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=BATCH_SIZE, num_workers=2)

In [ ]:
cnn = Vanilla_CNN(width=16).to("cuda:0")
regressor = Simple_regressor(16 * 8 * 8).to("cuda:0")

In [ ]:
params = list(cnn.parameters()) + list(regressor.parameters())
optim = torch.optim.Adam(params, lr=1e-4)

loss_fn = torch.nn.BCEWithLogitsLoss()

In [ ]:
cnn.train()
regressor.train()

for epoch in range(N_EPOCH):
    timer = time.time()

    img_processed = 0

    for img, target in train_loader:

        optim.zero_grad()

        img, target = img.to("cuda:0"), target.to("cuda:0")

        pred1 = cnn(img)
        pred2 = regressor(pred1.flatten(1, -1)).squeeze()

        loss = loss_fn(pred2, target.to(torch.float))

        loss.backward()

        optim.step()

        img_processed += img.shape[0]

    print("Epoch : {}".format(epoch + 1))
    print("Time elapsed : {}".format(time.time() - timer))

In [ ]:
# Accuracy

cnn.eval()
regressor.eval()

score = []

for img, target in test_loader:

    img, target = img.to("cuda:0"), target.to("cuda:0")

    pred1 = cnn(img)
    pred2 = regressor(pred1.flatten(1, -1)).squeeze()

    score_ = ((pred2 > 0.5) == (target == 1)).sum()

    score.append(score_)

score = torch.Tensor(score).sum() / len(test_loader.dataset)
print(score)